In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
import datajoint as dj
import calcium as ca

In [ ]:
schema = dj.schema('dln_YOUR_USERNAME_analysis')

In [ ]:
dj.Di(ca)

In [ ]:
combined = ca.Pupil * ca.Treadmill * ca.ScanInfo

In [ ]:
key = combined.fetch('KEY')[0]
r, v, f =(combined & key).fetch1('pupil_r', 'treadmill_vel', 'fps', squeeze=True)
t = np.r_[:r.size]/f

plt.plot(t, r)
plt.plot(t, abs(v)*10)

plt.legend(('pupil radius', 'running speed'));


In [ ]:
@schema
class PupilCorr(dj.Computed):
    definition = """
    -> ca.Pupil
    -> ca.Treadmill 
    -> ca.ScanInfo
    ---
    pupil_corr :  float   # correlation between pupil radius and locomotion velccity
    """
    
    def make(self, key):
        r, v, f =(ca.Pupil * ca.Treadmill * ca.ScanInfo & key).fetch1(
            'pupil_r', 'treadmill_vel', 'fps', squeeze=True)
        t = np.r_[:r.size]/f
        v = abs(v)
        v = v- np.nanmean(v, keepdims=True)
        r = r- np.nanmean(r, keepdims=True)
        r = r / np.sqrt(np.nanmean(r**2))
        v = v / np.sqrt(np.nanmean(v**2))
        c = np.nanmean(r * v)
        self.insert1(dict(key, pupil_corr=c))
        

In [ ]:
PupilCorr.key_source

In [ ]:
PupilCorr.populate(display_progress=True, suppress_errors=False,
                   reserve_jobs=True)

In [ ]:
PupilCorr()